### Import packages

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
from urllib.request import urlopen
from tqdm import tqdm
import random
import time
from selenium import webdriver

### Global variables

In [9]:
categories = {'sport': 'adventure-and-sport',
             'music': 'music',
             'film': 'film-and-video',
             'theatre': 'live-performance'}

### Functions

In [2]:
# Create an url
def create_url(page, category):
    _filter = '&categories[' + categories[category] + ']=on'
    return 'https://www.kisskissbankbank.com/en/discover?project[successful]=on' + _filter + '&page=' + str(page)

In [3]:
# Get a page from url
def get_page(url):
    ok = False
    page = requests.get(url)
    time.sleep(5)
    page = requests.get(url)
    if(page.status_code == 200):
        ok = True
    bs = BeautifulSoup(page.text, 'html.parser')
    return bs, ok

In [4]:
# Get the number of the projects from bs
def get_num_of_projects(bs):
    total_projects = bs.findAll('span', 'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-size-tiny k-u-weight-regular')
    return int(total_projects[0].contents[0].split(' ')[0].replace(',','')) 

In [25]:
# Get the number of pages from bs
def get_num_of_pages(bs):
    num_pro = get_num_of_projects(bs)
    return int(num_pro/9) + 1

In [89]:
# Function to go to project page and get information
def go_to_project(url_pro):
    
    # Load the website by driver
#     chrome_path = './chromedriver.exe'
    chrome_path = r'D:\\Telecom 1e anne\\Telecom Period 1\\Jupyter\\MODS203\\Project\\chromedriver.exe'
    driver = webdriver.Chrome(chrome_path)
    driver.get(url_pro)
    time.sleep(5)
    
    # Accept the cookie
    cookie_question = driver.find_element_by_xpath("//*[@id=\"App-react-component\"]/div[1]/div/div/div/div/button[2]")
    cookie_question.click()
    
    # Load the html page
    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    
    # Calculate num_news, num_comments, num_contributions
    infos = bs.findAll('a', 'k-NavBar__link')
    news_bs = infos[2].findAll('span', 'badge__StyledBadge-sc-7liuod-0 eucluz k-Badge k-Badge--spaced')
    comments_bs = infos[3].findAll('span', 'badge__StyledBadge-sc-7liuod-0 eucluz k-Badge k-Badge--spaced')
    contributions_bs = infos[4].findAll('span', 'badge__StyledBadge-sc-7liuod-0 eucluz k-Badge k-Badge--spaced')
    
    if(len(news_bs) == 0):
        num_news = 0
    else:
        num_news = int(news_bs[0].contents[0])

    if(len(comments_bs) == 0):
        num_comments = 0
    else:
        num_comments = int(comments_bs[0].contents[0])

    if(len(contributions_bs) == 0):
        num_contributions = 0
    else:
        num_contributions = int(contributions_bs[0].contents[0])
    
    # Calculate num_rewards, min_price, max_price, and backers each price
    rewards = bs.findAll('div','marger__StyledMarger-sc-1qqifp5-0 kXGmDY')
    prices = []
    backers = []
    for j in range(len(rewards)):
        rj = rewards[j]
        price_j = rj.findAll('h2', 'title__StyledTitle-sc-46lshq-0 edmigd k-RewardCard__title k-u-margin-bottom-double k-u-margin-top-none')
        if(len(price_j) == 0):
            break
        price_j = float(price_j[0].contents[0].replace('€','').replace(',',''))
        prices.append(price_j)
        backers_j = rj.findAll('span', 'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-size-micro k-u-weight-regular k-RewardCard__infos k-RewardCard__infos--hasBottomMargin k-RewardCard__infos--disabled')
        bj = 0
        if(len(backers_j) != 0 and backers_j[0].contents[0].replace(' ','').isnumeric()):
            bj = int(backers_j[0].contents[0])
        backers.append(bj)
    prices_np = np.array(prices)
    backers_np = np.array(backers)
    if(len(prices_np) == 0):
        min_price = -1
        max_price = -1
    else:
        min_price = np.min(prices_np)
        max_price = np.max(prices_np)
    num_rewards = len(prices_np)
    
#     sub_cats = driver.find_element_by_xpath('//*[@class="text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-size-micro k-u-weight-regular k-CrowdfundingCard__subtitle__subtitleText k-u-margin-none"]')
#     name = #
#     link = #
#     lang = #
#     text = #
    #print(backers_np)
    
    # Calculate num_creator_projects
    url_owner = bs.findAll('a', 'owner-info__StyledOwnerGrid-tqxc8c-0 jnrkmW')[0].get('href')
    driver_owner = webdriver.Chrome(chrome_path)
    driver_owner.get('https://www.kisskissbankbank.com'+ url_owner)
    time.sleep(5)
    cookie_question_owner = driver_owner.find_element_by_xpath('//*[@id="Header-react-component"]/div[1]/div/div/div/div/button[2]')
    cookie_question_owner.click()
    html_owner = driver_owner.page_source
    bs_owner = BeautifulSoup(html_owner, 'html.parser')
    num_creator_projects = int(bs_owner.findAll('span', 'k-Badge k-HorizontalNav__badge')[0].contents[0])
    #print(num_creator_projects)
    
    driver_owner.quit()
    driver.quit()
    return num_news, num_comments, num_contributions, num_rewards, min_price, max_price, prices_np, backers_np, num_creator_projects

In [90]:
go_to_project('https://www.kisskissbankbank.com/en/projects/3-8-premier-album-de-cvn-le-groupe-de-virginie-et-nico-bravin')

(15,
 66,
 161,
 7,
 15.0,
 2500.0,
 array([  15.,   25.,   40.,   45.,   75., 1000., 2500.]),
 array([54, 27, 11, 10, 16,  1,  0]),
 1)

In [74]:
chrome_path = r'D:\\Telecom 1e anne\\Telecom Period 1\\Jupyter\\MODS203\\Project\\chromedriver.exe'
driver = webdriver.Chrome(chrome_path)
driver.get('https://www.kisskissbankbank.com/en/discover?project[successful]=on&categories[music]=on&page=1')
html = driver.page_source
bs = BeautifulSoup(html, 'html.parser')

In [91]:
a = driver.find_element_by_xpath('//*[@class="text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-size-micro k-u-weight-regular k-CrowdfundingCard__subtitle__subtitleText k-u-margin-none"]')
a.text

'Music, Recording'

In [92]:
b = a.text
b.split()

['Music,', 'Recording']

In [80]:
print(html)

<html lang="en"><head>
<meta data-react-helmet="true" name="mobile-web-app-capable" content="yes"><meta data-react-helmet="true" name="apple-mobile-web-app-capable" content="yes"><meta data-react-helmet="true" name="apple-mobile-web-app-title" content="KissKissBankBank"><meta data-react-helmet="true" name="theme-color" content="#19b4fa"><meta data-react-helmet="true" name="msapplication-navbutton-color" content="#19b4fa"><meta data-react-helmet="true" name="apple-mobile-web-app-status-bar-style" content="#19b4fa"><meta data-react-helmet="true" name="msapplication-starturl" content="/"><meta data-react-helmet="true" charset="utf-8"><meta data-react-helmet="true" property="author" content="KissKissBankBank"><meta data-react-helmet="true" property="og:site_name" content="KissKissBankBank"><meta data-react-helmet="true" property="og:url" content="https://www.kisskissbankbank.com/en/discover"><meta data-react-helmet="true" property="og:locale" content="en"><meta data-react-helmet="true" pro

In [46]:
# Function to collect all features
def collect_data(start = 1, end = 2, category = 'music'):
    samples = []
    reward_prices = []
    count = (start - 1)*9
    for i in tqdm(range(start,end)):
        url_i = create_url(i,category)
        #print(url_i)
        #bs_i, ok = get_page(url_i)
        # Load the website by driver - First time
#         chrome_path = './chromedriver.exe'
        chrome_path = r'D:\\Telecom 1e anne\\Telecom Period 1\\Jupyter\\MODS203\\Project\\chromedriver.exe'
        driver_i = webdriver.Chrome(chrome_path)
        driver_i.get(url_i)
        time.sleep(10)

        # Accept the cookie
        cookie_question_i = driver_i.find_element_by_xpath("//*[@id=\"App-react-component\"]/div[1]/div/div/div/div/button[2]")
        cookie_question_i.click()

        # Load the html page
        html_i = driver_i.page_source
        bs_i = BeautifulSoup(html_i, 'html.parser')
        cats = driver_i.find_element_by_xpath('//*[@class="text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-size-micro k-u-weight-regular k-CrowdfundingCard__subtitle__subtitleText k-u-margin-none"]').text.split()
        
        projects = bs_i.findAll('a', 'styles__StyledCrowdfundingCard-sc-1dxuhb7-0 dOcwdr k-CrowdfundingCard k-Card k-Card--light k-Card--withoutBoxShadowOnHover k-CrowdfundingCard--titlesMinHeight')
        for p in tqdm(projects):
            # Extract is_successful
            is_successful = p.findAll('span','state-badge__StyledState-sc-1ixvpgd-0 efdutF')[0].contents[1]

            # Extract percentage_fund
            percentage_fund = p.findAll('span', 'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-size-micro k-u-weight-regular k-CrowdfundingCard__progressBar__percent k-u-hidden@s-down')[0].contents[0]

            # Extract funded, target, in_2020, backers
            infos = p.findAll('div', 'k-CrowdfundingCard__informations__infoContainer')
            funded = infos[2].findAll('strong', 'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-weight-regular info__StyledText-h7ofxb-0 jfWZBp')[0].contents[0]
            target = infos[2].findAll('span', 'k-u-hidden@s-down')[0].contents[0]
            in_2020 = infos[1].findAll('strong', 'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-weight-regular info__StyledText-h7ofxb-0 jfWZBp')[0].contents[0]
            backers = p.findAll('strong', 'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-weight-regular info__StyledText-h7ofxb-0 jfWZBp')[0].contents[0]

            # Extract category

            # Extract thumbnail type
            img = p.findAll('div','k-CrowdfundingCard__image__imageContainer')[0]
            thumbnail_type = img.find('img').get('src').split('.')[-1]

            # Extract remaining features
            url_pro = p.get('href')
            num_news, num_comments, num_contributions, num_rewards, min_price, max_price, prices_np, backers_np, num_creator_projects = go_to_project(url_pro)

            #print(num_news, num_comments, num_contributions,num_rewards, min_price, max_price)
            #print(backers_np)
            #print(prices_np)
            #print(num_creator_projects)
            #print('----------------------------------')
            sample_i = [
                count,
                is_successful,
                percentage_fund,
                funded,
                target,
                in_2020,
                backers,
                category,
                thumbnail_type,
                num_rewards,
                min_price,
                max_price,
                num_news,
                num_comments,
                num_contributions,
                num_creator_projects
            ]
            print(sample_i)
            for j in range(len(prices_np)):
                rp = [count, category, prices_np[j], backers_np[j]]
                reward_prices.append(rp)
            samples.append(sample_i)
            count += 1
            time.sleep(random.random()*2)
        driver_i.quit()
        time.sleep(random.random()*5)
    return samples, reward_prices


### Tests

In [34]:
url = create_url(2,'music')
bs, ok = get_page(url)
url, ok

('https://www.kisskissbankbank.com/en/discover?project[successful]=on&categories[music]=on&page=2',
 True)

In [35]:
get_num_of_projects(bs)

5715

In [36]:
num_pages = get_num_of_pages(bs)
num_pages

636

In [51]:
data, rewards = collect_data()

  0%|                                                                                            | 0/1 [02:25<?, ?it/s]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="App-react-component"]/div[1]/div/div/div/div/button[2]"}
  (Session info: chrome=86.0.4240.111)


In [44]:
chrome_path = r'D:\\Telecom 1e anne\\Telecom Period 1\\Jupyter\\MODS203\\Project\\chromedriver.exe'
driver = webdriver.Chrome(chrome_path)

### Main

In [44]:
features_data = [
    'id',
    'is_successful',
    'percentage_fund',
    'funded',
    'target',
    'in_2020',
    'backers',
    'category',
    'thumbnail_type',
    'num_rewards',
    'min_price',
    'max_price',
    'num_news',
    'num_comments',
    'num_contributions',
    'num_creator_projects'
]
features_rewards = [
    'id',
    'category',
    'price',
    'backers'
]

In [41]:
data, rewards = collect_data(1,12,'theatre')

  0%|          | 0/9 [00:00<?, ?it/s]

[0, 'successful', '100 %', '€4,005', 'Out of €4,000', '12/16/2020', '15', 'theatre', 'jpg', 9, 5.0, 600.0, 0, 3, 18, 1]



 11%|█         | 1/9 [00:37<04:56, 37.03s/it]

[1, 'successful', '131 %', '€7,859', 'Out of €6,000', '12/16/2020', '40', 'theatre', 'jpeg', 7, 20.0, 500.0, 0, 6, 43, 1]



 22%|██▏       | 2/9 [01:13<04:18, 36.88s/it]

...


 78%|███████▊  | 7/9 [04:21<01:14, 37.08s/it]

[97, 'successful', '103 %', '€3,085', 'Out of €3,000', '4/11/2019', '90', 'theatre', 'jpg', 7, 10.0, 1000.0, 3, 22, 91, 1]



 89%|████████▉ | 8/9 [04:58<00:36, 36.89s/it]

[98, 'successful', '118 %', '€21,150', 'Out of €18,000', '4/8/2019', '164', 'theatre', 'png', 5, 20.0, 500.0, 2, 50, 181, 1]



100%|██████████| 11/11 [1:06:13<00:00, 361.20s/it]


In [29]:
data_df = pd.DataFrame(data, columns = features_data)
data_df

NameError: name 'data' is not defined

In [28]:
rewards_df = pd.DataFrame(rewards, columns = features_rewards)
rewards_df

NameError: name 'rewards' is not defined

In [51]:
data_df.to_csv('./data/data_theatre_1_11.csv', index=False)
rewards_df.to_csv('./data/rewards_theatre_1_11.csv', index=False)